# NRM python upstream client library tutorial

This tutorial covers the use of NRM's python upstream client library, in the context of running an external resource management strategy. Its cell's output are deterministic, and the executed version that is vendored in the source tree is checked by the project's CI, so its behavior should always be up-to-date with the latest version of the software, and no cells should be throwing exceptions.  

The next few cells are for setup purposes.

In [1]:
%%capture
cd ..

In [2]:
%load_ext nb_black

<IPython.core.display.Javascript object>

## Setup

This notebook uses `nrm`'s python library bindings and needs the `nrmd` daemon in the `$PATH`. 

Assuming the project is cloned **with submodules updated** (and the code unmodified), one needs to run the following from the root of the project before running it:

In [3]:
%%capture
%%bash
./shake.sh build # for the daemon 
./shake.sh client # for the upstream client
./shake.sh pyclient # for the shared client library

<IPython.core.display.Javascript object>

This allows to use the command-line daemon and client in the working directory, should you need to for debugging purposes.

In [6]:
%%bash
nrmd --help

nrmd

Usage: <unknown> [-i|--stdin] [CONFIG] [-y|--yaml] [-e|--edit]
  NRM Daemon

Available options:
  -h,--help                Show this help text
  -i,--stdin               Read configuration on stdin.
  CONFIG                   Input configuration with .yml/.yaml/.dh/.dhall
                           extension. Leave void for stdin (dhall) input.
  -y,--yaml                Assume configuration to be yaml instead of dhall.
  -e,--edit                Edit yaml in $EDITOR before running the NRM daemon.
  -h,--help                Show this help text


<IPython.core.display.Javascript object>

In [7]:
%%bash
nrm --help

nrm

Usage: nrm COMMAND
  NRM Client

Available options:
  -h,--help                Show this help text
  COMMAND                  Choice of operation.

Available commands:
  run                      Run the application via NRM
  killcmd                  Kill cmd
  killslice                Kill slice
  setpower                 Set power limit
  cpd                      Show current CPD
  list                     List existing slices
  state                    Show NRM state
  config                   Show NRM configuration
  listen-raw               Listen to raw NRM upstream pub messages
  listen-cpd               Listen to CPD messages
  listen-all               Listen to all upstream pub messages


<IPython.core.display.Javascript object>

## Defining experiments

Now that the daemon is properly set-up, we will configure and run some experiments using the python interface.

The next cell imports the upstream client library. and configures hosts.

In [8]:
from nrm.tooling import Local, Remote, lib

<IPython.core.display.Javascript object>

This notebook will start `nrmd` on the same machine as the notebok, but the same interface should be available for remote execution:

In [9]:
host = Local()
# host=Remote( target="cc@129.114.108.201")

<IPython.core.display.Javascript object>

Note that the two classes (`Local` and `Remote`) offer the same methods to start nrmd and interact via blocking message passing primitives. The supported methods are the following:

In [10]:
import inspect

for a, x in inspect.getmembers(host, predicate=inspect.ismethod):
    print("%s: %s" % (a, x.__doc__))

__init__: None
check_daemon:  checks if nrmd is alive 
run_workload:  Runs a workload via NRM. The `nrmd` daemon must be running. 
start_daemon:  start nrmd 
stop_daemon:  stops nrmd 
workload_exit_status:  Check the workload's exit status. 
workload_finished:  Checks NRM to see whether all tasks are finished. 
workload_recv:  Receive a message from NRM's upstream API. 
workload_send:  Send a message to NRM's upstream API. 


<IPython.core.display.Javascript object>

The next cell defines some node experimental node (`nrmd`) daemon configurations and some workloads. Please see notebook [notebooks/configuration.ipynb](notebooks/configuration.ipynb) for an overview of `nrmd`'s and the "Manifest"'s configuration format.

In [11]:
daemonCfgs = {
    "redirected_log": {"logfile": "/tmp/logfile_experiment2", "verbose": "info"},
    # "other":your code
}

workloads = {
    "dummy":{ "cmd":"sleep",
              "args": ["333"] ,
              "sliceID": "toto",
              "manifest":{"app": {
                              "slice": {
                                  "cpus": 1,
                                  "mems": 1 } 
                                  },
                          "name": "default"
                         },
            },
    #"other":your code
}

<IPython.core.display.Javascript object>

The next cell defines some experiments:

In [12]:
experiments = {
    "example": (daemonCfgs["redirected_log"], workloads["dummy"]),
    # "other": todo
}

<IPython.core.display.Javascript object>

## Running experiments
The next two cells show how to start and stop the daemon. A failure in either of them indicates a problem with this experimental NRM setup.

In [13]:
host.start_daemon(daemonCfgs["redirected_log"])
assert host.check_daemon()

<IPython.core.display.Javascript object>

In [14]:
host.stop_daemon()
assert host.check_daemon() == False

<IPython.core.display.Javascript object>

### Dummy experiment: printing measurements

We now are ready to run an external resource management strategy. Using the low-level message passing interface.

In [ ]:
for name, (daemonCfg, workload) in experiments.items():
    # starting the daemon (does act as silent restart)
    host.start_daemon(daemonCfg)
    # running the workload
    host.run_workload(workload)
    # message passting exchange:
    while not host.workload_finished():
        measurement_message = host.workload_recv()
        print(measurement_message)
        command_message = "insert your code here"
        host.workload_send(command_message)
    print(host.check_daemon())

host.stop_daemon()